In [1]:
import pandas as pd
import datetime as dt

In [2]:
ruta="/home/donsson/proyectos/VENTAS/Referencias Promo Autoparts.xlsx"
facturas = "/home/donsson/proyectos/API/ventashistoricas52semanas.parquet"

promos = pd.read_excel(ruta)

df_f = pd.read_parquet(facturas)

In [3]:
promos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Referencia interna  143 non-null    object
 1   Nombre              143 non-null    object
 2   Categoría interna   143 non-null    object
 3   Clase               143 non-null    object
 4   Precio de venta     143 non-null    int64 
 5   Cantidad a mano     143 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 6.8+ KB


In [4]:
promos["Referencia interna"].count()

np.int64(143)

In [5]:
df_f["Referencia interna"] = df_f["product_name"].str.extract(r"\[([^\]]+)\]")
df_f["Nombre"] = df_f["product_name"].str.replace(r"\[.*?\]\s*", "", regex=True)


In [6]:
df_f["ref"] = df_f["Referencia interna"].astype(str).str.strip().str.upper()
promos["ref"] = promos["Referencia interna"].astype(str).str.strip().str.upper()

# Hacemos inner join: solo los que están en la promo
df_promo_facturas = promos.merge(
    df_f,
    how="inner",
    on="ref",
    suffixes=("_promo", "_factura")
)

df_promo_facturas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16591 entries, 0 to 16590
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Referencia interna_promo    16591 non-null  object        
 1   Nombre_promo                16591 non-null  object        
 2   Categoría interna           16591 non-null  object        
 3   Clase                       16591 non-null  object        
 4   Precio de venta             16591 non-null  int64         
 5   Cantidad a mano             16591 non-null  int64         
 6   ref                         16591 non-null  object        
 7   origin                      16591 non-null  string        
 8   price_subtotal              16591 non-null  float64       
 9   quantity                    16591 non-null  float64       
 10  product_name                16591 non-null  object        
 11  invoice_name                16591 non-null  object    

In [7]:
df_promo_facturas_limpio = df_promo_facturas[[
    "ref",
    "Clase",
    "Nombre_promo",
    "date_invoice",
    "quantity",
    "price_subtotal",
    "Precio de venta"
]].copy()

# ✅ Paso 2. Convertimos a datetime (sin warnings)
df_promo_facturas_limpio.loc[:, "date_invoice"] = pd.to_datetime(df_promo_facturas_limpio["date_invoice"])

# ✅ Paso 3. Creamos columna Fecha solo con año-mes-día
df_promo_facturas_limpio["Fecha"] = df_promo_facturas_limpio["date_invoice"].dt.date

# ✅ Paso 4. Filtrar septiembre
inicio_promo = pd.to_datetime("2025-09-01")
fin_promo    = pd.to_datetime("2025-09-30")


# ✅ Paso 5. Agrupar por día + producto
df_resumen_dia = (df_promo_facturas_limpio
                  .groupby(["ref","Clase", "Nombre_promo", "Fecha"], as_index=False)
                  .agg({
                      "quantity": "sum",
                      "price_subtotal": "sum",
                      "Precio de venta":"mean"
                  }))


df_resumen_dia["Fecha"] = pd.to_datetime(df_resumen_dia["Fecha"], errors="coerce")

df_resumen_dia["mes"] = df_resumen_dia["Fecha"].dt.month
df_resumen_dia["año"] = df_resumen_dia["Fecha"].dt.year

# Columna de fecha tipo período mensual
df_resumen_dia["Periodo"] = pd.to_datetime(
    df_resumen_dia["año"].astype(str) + "-" + df_resumen_dia["mes"].astype(str) + "-01"
)

df_resumen_dia["Periodo"] = pd.to_datetime(df_resumen_dia["Periodo"], errors="coerce")


In [8]:
fecha_promocion = pd.to_datetime("2025-09-01")

df_resumen_dia["Etapa"] = df_resumen_dia["Periodo"].apply(
    lambda x: "Antes" if x < fecha_promocion else "Después"
)



df_resumen_dia.to_excel("promoseptiembreautoparts.xlsx")

In [9]:

comparacion = (
    df_resumen_dia.groupby(["Clase", "Etapa"], as_index=False)
              .agg({
                  "quantity": "mean",  # promedio de ventas
                  "price_subtotal": "mean"
              })
)


In [10]:
import plotly.express as px

fig = px.line(
    df_resumen_dia,
    x="Periodo", y="quantity",
    color="Clase",
    title="Evolución mensual de ventas por Clase"
)
fig.show()
